In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part_number'):
    shutil.rmtree('./file/2.part_number')

os.makedirs('./file/2.part_number')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:    
            output, i = pd.DataFrame(), 0
            while True:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter=-All-&filterType=n&searchType=p&imageSize=80&start={i*96}&limit=96&sort=3&catFilter={input_.loc[a, 'Type Code']}&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_json = resp.json()

                # = = = = = = = = = = = = = = =

                if not list_json:
                    raise

                # = = = = = = = = = = = = = = =

                if i == 0:
                    count = list_json[0]['maxRows']

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': [json_['rowId'] for json_ in list_json],
                                        'Part Number': [json_['basePart'] for json_ in list_json]})

                output = pd.concat([output, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
    
                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
    
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Type Code']} > Page {i+1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
                # = = = = = = = = = = = = = = =

                i += 1

                # = = = = = = = = = = = = = = =

                if len(output) == count:
                    output = output.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
                    
                    output.to_excel(f'''./file/2.part_number/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
                    
                    break
                    
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part_number/{file}',
              f'''./file/2.part_number/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[尝试次数：1] - 1471.138100 > Page 4
[剩余数量：100] - [当前时间：20:20:37]

[尝试次数：2] - 1465.122100 > Page 3
[剩余数量：99] - [当前时间：20:20:38]

[尝试次数：1] - 1473.140135 > Page 3
[剩余数量：99] - [当前时间：20:20:38]

[尝试次数：1] - 1474.139100 > Page 1
[剩余数量：99] - [当前时间：20:20:39]

[尝试次数：1] - 1467.59100 > Page 2
[剩余数量：98] - [当前时间：20:20:39]

[尝试次数：1] - 1475.144100 > Page 1
[剩余数量：98] - [当前时间：20:20:40]

[尝试次数：1] - 1473.140135 > Page 4
[剩余数量：98] - [当前时间：20:20:40]

[尝试次数：1] - 1465.122100 > Page 4
[剩余数量：97] - [当前时间：20:20:40]

[尝试次数：1] - 1476.276900 > Page 1
[剩余数量：97] - [当前时间：20:20:40]

[尝试次数：1] - 1467.59100 > Page 3
[剩余数量：96] - [当前时间：20:20:40]

[尝试次数：1] - 1458.111100 > Page 7
[剩余数量：95] - [当前时间：20:20:41]

[尝试次数：1] - 1475.144100 > Page 2
[剩余数量：95] - [当前时间：20:20:41]

[尝试次数：1] - 1478.151100 > Page 1
[剩余数量：94] - [当前时间：20:20:42]

[尝试次数：1] - 1465.122100 > Page 5
[剩余数量：94] - [当前时间：20:20:42]

[尝试次数：1] - 1480.294600 > Page 1
[剩余数量：94] - [当前时间：20:20:42]

[尝试次数：1] - 1458.111100 > Page 8
[剩余数量：93] - [当前时间：20:20:43]

[尝试次数：1] - 1479.153100 > 

Progress: 100%|███████████████████████| 1574/1574 [00:00<00:00, 10504.73it/s]

Done ~
